In [1]:
import pandas as pd

df = pd.read_csv("C:\\Users\\DELL\\OneDrive\\Área de Trabalho\\Ironhack\\final project\\data\\cleaned\\model.csv")
df = df.sort_values("data")

In [2]:
X = df.drop(columns=["target", "selic_next", "data"])

y = df["target"]

In [4]:
split = int(len(df) * 0.8)

X_train = X.iloc[:split]
X_test  = X.iloc[split:]

y_train = y.iloc[:split]
y_test  = y.iloc[split:]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=500)
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=500)

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.5652173913043478
[[12  0]
 [10  1]]
              precision    recall  f1-score   support

           0       0.55      1.00      0.71        12
           1       1.00      0.09      0.17        11

    accuracy                           0.57        23
   macro avg       0.77      0.55      0.44        23
weighted avg       0.76      0.57      0.45        23



In [8]:
import numpy as np

coef = model.coef_[0]
features = X.columns

importance_df = pd.DataFrame({
    "feature": features,
    "coef": coef,
    "abs_coef": np.abs(coef)
}).sort_values("abs_coef", ascending=False)

importance_df.head(15)


,feature,coef,abs_coef
16,unemployment_pct3,-0.982413,0.982413
26,ipca_lag3,0.947279,0.947279
12,debt_dbgg_pct3,-0.878640,0.878640
15,unemployment_pct1,-0.824069,0.824069
11,debt_dbgg_pct1,-0.666265,0.666265
30,unemployment_lag3,0.651241,0.651241
31,usd_brl_lag1,0.574602,0.574602
20,ibc_br_std3,0.541814,0.541814
6,usd_brl,0.539047,0.539047
4,selic_efetiva,0.498471,0.498471


In [ ]:
# New approach

In [12]:
df["selic_target"] = df["selic_next"]

In [25]:
# Drop columns that are not useful or leak information
df_reg = df.drop(columns=["data", "target"])  # vamos remover também o target antigo

# Define X and y
X = df_reg.drop(columns=["selic_next", "selic_target"])
y = df_reg[["selic_next", "selic_target"]]

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, shuffle=False
)


In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
# 1. Linear Regression
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

LinearRegression()

In [29]:
# 2. Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=300, random_state=42)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=300, random_state=42)

In [30]:
# 3. XGBoost Regressor

from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.03, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, ...)

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    return mae, rmse, r2

for name, model in [("Linear", lr), ("RandomForest", rf), ("XGB", xgb)]:
    mae, rmse, r2 = evaluate(model, X_test_scaled, y_test)
    print(f"=== {name} ===")
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²: {r2:.3f}\n")


=== Linear ===
MAE: 0.272
RMSE: 0.313
R²: 0.966

=== RandomForest ===
MAE: 0.598
RMSE: 0.715
R²: 0.820

=== XGB ===
MAE: 0.681
RMSE: 0.802
R²: 0.774



In [35]:
pred_test = rf.predict(X_test_scaled)
test_class = (pred_test > X_test["selic_meta"].values).astype(int)

pred_test[:10], y_test[:10]

ValueError: operands could not be broadcast together with shapes (23,2) (23,) 